In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using `torch.load` with `weights_only=False`")
warnings.filterwarnings("ignore", category=UserWarning, message="The PyTorch API of nested tensors is in prototype stage and will change in the near future.")
%load_ext autoreload
%autoreload 2

import sys, os
from moonshot_models.SMART_BART import SmartBart
from moonshot_dataset.NMR_selfie_dataset import NMR_Selfie_DataModule, SELFIES_MAX_LEN
from train_SMART_BART import data_mux
from pathlib import Path
import torch
# from torch.utils.data import DataLoader
torch.set_float32_matmul_precision('medium')

In [2]:
model_class = SmartBart
import yaml
checkpoint_path = "/root/gurusmart/Moonshot/give_up_multiple_schedulers/compare_unfreeze/only_hsqc_1_from_scratch/checkpoints/epoch=97-step=83986.ckpt"
checkpoint_path = Path(checkpoint_path)
model_path = checkpoint_path.parents[1]
hyperpaerameters_path = model_path / "hparams.yaml"

with open(hyperpaerameters_path, 'r') as file:
    args = yaml.safe_load(file)
data_module = data_mux(args)

del args['checkpoint_path']
args['validate_with_generation'] = True

In [3]:

model = model_class.load_from_checkpoint(checkpoint_path=checkpoint_path, 
                                                 selfie_symbol_to_idx = data_module.symbol_to_idx,
                                                selfie_max_len = 600,
                                                p_args = args,
        )

Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


load_encoder False


In [7]:
model = model.to('cpu')

In [8]:
import pytorch_lightning as pl

trainer = pl.Trainer(logger=None,  use_distributed_sampler=False)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
data_module.setup(stage='predict')

In [15]:
model.eval()
for batch in data_module.predict_dataloader():
    # (inputs, NMR_type_indicator, encoded_selfie, smiles ,mfp) = batch
    batch[0] = batch[0].to(model.device)
    batch[1] = batch[1].to(model.device)
    result = model.predict_step(batch, 0)
    break
    

tensor([False])
next_token [START]
tensor([False])
next_token [C]
tensor([False])
next_token [C]
tensor([False])
next_token [C]
tensor([False])
next_token [Branch1]
tensor([False])
next_token [C]
tensor([False])
next_token [C]
tensor([False])
next_token [C]
tensor([False])
next_token [=C]
tensor([False])
next_token [C]
tensor([False])
next_token [=C]
tensor([False])
next_token [C]
tensor([False])
next_token [=C]
tensor([False])
next_token [Branch1]
tensor([False])
next_token [C]
tensor([False])
next_token [C]
tensor([False])
next_token [C]
tensor([False])
next_token [O]
tensor([False])
next_token [C]
tensor([False])
next_token [C]
tensor([False])
next_token [=C]
tensor([False])
next_token [N]
tensor([False])
next_token [Branch1]
tensor([False])
next_token [Ring1]
tensor([False])
next_token [O]
tensor([False])
next_token [C]
tensor([False])
next_token [C]
tensor([False])
next_token [=Branch1]
tensor([False])
next_token [C]
tensor([False])
next_token [=O]
tensor([False])
next_token [C]
t

In [12]:
result

{'truth_smiles': ['CCC(C)C=CC=CC=C(C)C1Oc2cc[nH]c(=O)c2C(O)C1C'],
 'predicted_smiles': ['CCC(C)C=CC=CC=C(C1)COC=2C=CN(OC)C(=O)C=2C1(C)O'],
 'cosine_similarity': tensor(0.4386),
 'valid_smiles_rate': tensor(1.)}

In [ ]:
batch[2]

tensor([[  1, 201, 201, 201, 188, 201, 201, 201, 118, 201, 118, 201, 118, 188,
         201, 201, 201, 316, 201, 201, 118, 293, 201, 111, 201, 138, 201, 142,
           7, 201, 188, 201, 316, 201, 333, 297, 201,   2,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0

In [ ]:
model.idx_to_symbol[2]

'[END]'

In [ ]:
model.selfie_symbol_to_idx['[END]']

2